In [ ]:
from sklearn import svm
import numpy as np
from random import shuffle
from sklearn.model_selection import train_test_split
import math
import collections
from sklearn.model_selection import cross_val_score
import time

In [ ]:
file = open("glass.data","r")
trainX=np.zeros((214,9))
trainY=np.zeros((214,))
for i,line in enumerate(file.readlines()):
    splitting = line.split(",")
    trainX[i,:]=splitting[1:(len(splitting)-1)]
    trainY[i,]=splitting[len(splitting)-1]

In [ ]:
def linear_svc(Cs,train_set,test_set,train_label,test_label,decision_f_shape,weight=None):
    acc=0
    param={i:[]}
    start=time.time()
    for c in Cs:
        
        s=svm.SVC(kernel='linear',C=c,decision_function_shape=decision_f_shape,class_weight=weight)
        s.fit(train_set,train_label)
        accuracy=s.score(test_set, test_label)
        #print("Testing Hyperparmeter C for linear is ",c ," with accuracy", accuracy )
        if(accuracy>acc):
            acc=accuracy
            param[i]=[c]
    end=time.time()
    diffTime=end-start
    return acc,param,diffTime
        
    

In [ ]:
def sigmoid_svc(Cs,gammas,coeff,train_set,test_set,train_label,test_label,decision_f_shape,weight=None):
    acc=0
    param={i:[]}
    start=time.time()
    for c in Cs:
        for g in gammas:
            for r in coeff:
                s=svm.SVC(kernel='sigmoid',gamma=g,C=c,coef0=r,decision_function_shape='ovo',class_weight=weight)
                s.fit(train_set,train_label)
                accuracy=s.score(test_set, test_label)
                #print("Testing Hyperparmeter C, gamma, coef0 for sigmoid is ",c," , ",g," , ",r," with accuracy",accuracy)
                if(accuracy>acc):
                    acc=accuracy
                    param[i]=[c,g,r]
    diffTime=time.time()-start
    return acc,param,diffTime

In [ ]:
def rbf_svc(Cs,gammas,train_set,test_set,train_label,test_label,decision_f_shape,weight=None):
    acc=0
    param={i:[]}
    start=time.time()
    for c in Cs:
        for g in gammas:
            s=svm.SVC(kernel='rbf',gamma=g,C=c,decision_function_shape='ovo',class_weight=weight)
            s.fit(train_set,train_label)
            accuracy=s.score(test_set, test_label)
            #print("Testing Hyperparmeter C, gamma, coef0 for rbf is ",c," , ",g, "with accuracy", accuracy )
            if(accuracy>acc):
                acc=accuracy
                param[i]=[c,g]
    diffTime=time.time()-start
    return acc,param,diffTime

In [ ]:
def poly_svc(Cs,gammas,coeff,degr,train_set,test_set,train_label,test_label,decision_f_shape,weight=None):
    acc=0
    param={i:[]}
    start=time.time()
    for c in Cs:
        for g in gammas:
            for r in coeff:
                for d in degr:
                    s=svm.SVC(kernel='poly',gamma=g,C=c,coef0=r,degree=d,decision_function_shape='ovo',class_weight=weight)
                    s.fit(train_set,train_label)
                    accuracy=s.score(test_set, test_label)
                   # print("Testing Hyperparmeter C, gamma, coef0  and degree for polynomial is ",c," , ",g," , ",r ," , ",d,"  with accuracy ", accuracy )
                    if(accuracy>acc):
                        acc=accuracy
                        param[i]=[c,g,r,d]
    diffTime=time.time()-start
    return acc,param,diffTime

# One Vs One

In [ ]:

ind_list = [i for i in range(214)]
shuffle(ind_list)
train= trainX[ind_list,:]
train_label = trainY[ind_list,]
setSize=(int)(train.shape[0]/5)
f_Accuracy={'linear':[],'rbf':[],'sigmoid':[],'poly':[]}
for i in range(0,5):
    test_set=train[i*setSize:(i+1)*setSize,:]
    test_label=train_label[i*setSize:(i+1)*setSize,]
    train_set=np.delete(train,slice(i*setSize,(i+1)*setSize),0)
    train_lbl=np.delete(train_label,slice(i*setSize,(i+1)*setSize,))
    X_train, X_test, y_train, y_test = train_test_split(train_set, train_lbl, test_size=0.20)
    acc_param=[]
    kernels=['linear','sigmoid','rbf','poly']
    Cs = [math.pow(2,-5),math.pow(2,-3),math.pow(2,-1), math.pow(2,0),math.pow(2,1),math.pow(2,3)]
    gammas = [math.pow(2,-15), math.pow(2,-13),math.pow(2,-11),math.pow(2,-9),math.pow(2,-7),math.pow(2,-5),math.pow(2,-3)]
    degree=[2,3,4]
    coeff=[0,0.1,0.01,0.001]
    


    decision_function_shape='ovo'

    for k in kernels:
        if k=='linear':
            max_acc,optimal_param,diffTime1=linear_svc(Cs,X_train,X_test,y_train,y_test,decision_function_shape)

            acc,param,diffTime2=linear_svc(optimal_param[i],train_set,test_set,train_lbl,test_label,decision_function_shape)
            print("Time taken to find optimal hyperparameters is " ,diffTime1," seconds")
            print("Total time taken i.e. optimal hyperparam plus training using those optimal hyperparameters is ",diffTime1+diffTime2," seconds")
            print("Final Accuracy for fold",i+1," is ",acc," Using optimal hyperparameter C as ", optimal_param[i][0])
            print("\n")
            print("____________________________________")
            print("\n")
            f_Accuracy[k].append(acc)
        elif k=='sigmoid':
            Cs1 = [math.pow(2,-5),math.pow(2,-3),math.pow(2,-1), math.pow(2,0),math.pow(2,1),math.pow(2,3),math.pow(2,5), math.pow(2,8), math.pow(2,11), math.pow(2,13), math.pow(2,15)]
            gammas1 = [math.pow(2,-15), math.pow(2,-13),math.pow(2,-11)]
            max_acc,optimal_param,diffTime1=sigmoid_svc(Cs1,gammas1,coeff,X_train,X_test,y_train,y_test,decision_function_shape)
 
            cc=[optimal_param[i][0]]
            gg=[optimal_param[i][1]]
            rr=[optimal_param[i][2]]
            
            acc,param,diffTime2=sigmoid_svc(cc,gg,rr,train_set,test_set,train_lbl,test_label,decision_function_shape)
            print("Time taken to find optimal hyperparameters is " ,diffTime1," seconds")
            print("Total time taken i.e. optimal hyperparam plus training using those optimal hyperparameters is ",diffTime1+diffTime2," seconds")
            print("Final Accuracy for fold",i+1," is ",acc," Using Optimal hyperparameter C as ", optimal_param[i][0]," gamma as ",optimal_param[i][1],"and r as ",optimal_param[i][2])
            
            print("\n")
            print("____________________________________")
            print("\n")
            
            f_Accuracy[k].append(acc)
        elif k=="rbf":
            max_acc,optimal_param,diffTime1=rbf_svc(Cs,gammas,X_train,X_test,y_train,y_test,decision_function_shape)
  

            cc=[optimal_param[i][0]]
            gg=[optimal_param[i][1]]
            
            
            acc,param,diffTime2=rbf_svc(cc,gg,train_set,test_set,train_lbl,test_label,decision_function_shape)
            print("Time taken to find optimal hyperparameters is " ,diffTime1," seconds")
            print("Total time taken i.e. optimal hyperparam plus training using those optimal hyperparameters is ",diffTime1+diffTime2," seconds")
            print("Final Accuracy for fold",i+1," is ",acc," Using Optimal hyperparameter C as ", optimal_param[i][0]," gamma as ",optimal_param[i][1])
            print("\n")
            print("____________________________________")
            print("\n")

            f_Accuracy[k].append(acc)
        else:
            
            max_acc,optimal_param,diffTime1=poly_svc(Cs,gammas,coeff,degree,X_train,X_test,y_train,y_test,decision_function_shape)
            
          
            print("\n")
            print("____________________________________")
            print("\n")
            
            cc=[optimal_param[i][0]]
            gg=[optimal_param[i][1]]
            rr=[optimal_param[i][2]]
            dd=[optimal_param[i][3]]
            
            acc,param,diffTime2=poly_svc(cc,gg,rr,dd,train_set,test_set,train_lbl,test_label,decision_function_shape)
            print("Time taken to find optimal hyperparameters is " ,diffTime1," seconds")
            print("Total time taken i.e. optimal hyperparam plus training using those optimal hyperparameters is ",diffTime1+diffTime2," seconds")
            print("Final Accuracy for fold",i+1," is ",acc," Using Optimal hyperparameter C as ", param[i][0]," gamma as ",param[i][1]," r as ",param[i][2]," and degree as ",param[i][3])
            f_Accuracy[k].append(acc)
            
            
print("\n")
print("Final Accuracy for Linear is ",np.sum(f_Accuracy['linear'])/5)   
print("Final Accuracy for Sigmoid is ",np.sum(f_Accuracy['sigmoid'])/5)
print("Final Accuracy for RBF is ",np.sum(f_Accuracy['rbf'])/5)
print("Final Accuracy for Poly is ",np.sum(f_Accuracy['poly'])/5)
            

   

# One vs Rest

In [ ]:

ind_list = [i for i in range(214)]
shuffle(ind_list)
train= trainX[ind_list,:]
train_label = trainY[ind_list,]
setSize=(int)(train.shape[0]/5)
f_Accuracy={'linear':[],'rbf':[],'sigmoid':[],'poly':[]}
for i in range(0,5):
    test_set=train[i*setSize:(i+1)*setSize,:]
    test_label=train_label[i*setSize:(i+1)*setSize,]
    train_set=np.delete(train,slice(i*setSize,(i+1)*setSize),0)
    train_lbl=np.delete(train_label,slice(i*setSize,(i+1)*setSize,))
    X_train, X_test, y_train, y_test = train_test_split(train_set, train_lbl, test_size=0.20)
    acc_param=[]
    kernels=['linear','sigmoid','rbf','poly']
    Cs = [math.pow(2,-5),math.pow(2,-3),math.pow(2,-1), math.pow(2,0),math.pow(2,1),math.pow(2,3)]
    gammas = [math.pow(2,-15), math.pow(2,-13),math.pow(2,-11),math.pow(2,-9),math.pow(2,-7),math.pow(2,-5),math.pow(2,-3)]
    degree=[2,3,4]
    coeff=[0,0.1,0.01,0.001]
    decision_function_shape='ovr'

    for k in kernels:
        if k=='linear':
            max_acc,optimal_param,diffTime1=linear_svc(Cs,X_train,X_test,y_train,y_test,decision_function_shape)

            acc,param,diffTime2=linear_svc(optimal_param[i],train_set,test_set,train_lbl,test_label,decision_function_shape)
            print("Time taken to find optimal hyperparameters is " ,diffTime1," seconds")
            print("Total time taken i.e. optimal hyperparam plus training using those optimal hyperparameters is ",diffTime1+diffTime2," seconds")
            print("Final Accuracy for fold",i+1," is ",acc," Using optimal hyperparameter C as ", optimal_param[i][0])
            print("\n")
            print("____________________________________")
            print("\n")
            f_Accuracy[k].append(acc)
        elif k=='sigmoid':
            Cs1 = [math.pow(2,-5),math.pow(2,-3),math.pow(2,-1), math.pow(2,0),math.pow(2,1),math.pow(2,3),math.pow(2,5), math.pow(2,8), math.pow(2,11), math.pow(2,13), math.pow(2,15)]
            gammas1 = [math.pow(2,-15), math.pow(2,-13),math.pow(2,-11)]
            max_acc,optimal_param,diffTime1=sigmoid_svc(Cs1,gammas1,coeff,X_train,X_test,y_train,y_test,decision_function_shape)
 
            cc=[optimal_param[i][0]]
            gg=[optimal_param[i][1]]
            rr=[optimal_param[i][2]]
            
            acc,param,diffTime2=sigmoid_svc(cc,gg,rr,train_set,test_set,train_lbl,test_label,decision_function_shape)
            print("Time taken to find optimal hyperparameters is " ,diffTime1," seconds")
            print("Total time taken i.e. optimal hyperparam plus training using those optimal hyperparameters is ",diffTime1+diffTime2," seconds")
            print("Final Accuracy for fold",i+1," is ",acc," Using Optimal hyperparameter C as ", optimal_param[i][0]," gamma as ",optimal_param[i][1],"and r as ",optimal_param[i][2])
            
            print("\n")
            print("____________________________________")
            print("\n")
            
            f_Accuracy[k].append(acc)
        elif k=="rbf":
            max_acc,optimal_param,diffTime1=rbf_svc(Cs,gammas,X_train,X_test,y_train,y_test,decision_function_shape)
  

            cc=[optimal_param[i][0]]
            gg=[optimal_param[i][1]]
            
            
            acc,param,diffTime2=rbf_svc(cc,gg,train_set,test_set,train_lbl,test_label,decision_function_shape)
            print("Time taken to find optimal hyperparameters is " ,diffTime1," seconds")
            print("Total time taken i.e. optimal hyperparam plus training using those optimal hyperparameters is ",diffTime1+diffTime2," seconds")
            print("Final Accuracy for fold",i+1," is ",acc," Using Optimal hyperparameter C as ", optimal_param[i][0]," gamma as ",optimal_param[i][1])
            print("\n")
            print("____________________________________")
            print("\n")

            f_Accuracy[k].append(acc)
        else:
            
            max_acc,optimal_param,diffTime1=poly_svc(Cs,gammas,coeff,degree,X_train,X_test,y_train,y_test,decision_function_shape)
            
          
            print("\n")
            print("____________________________________")
            print("\n")
            
            cc=[optimal_param[i][0]]
            gg=[optimal_param[i][1]]
            rr=[optimal_param[i][2]]
            dd=[optimal_param[i][3]]
            
            acc,param,diffTime2=poly_svc(cc,gg,rr,dd,train_set,test_set,train_lbl,test_label,decision_function_shape)
            print("Time taken to find optimal hyperparameters is " ,diffTime1," seconds")
            print("Total time taken i.e. optimal hyperparam plus training using those optimal hyperparameters is ",diffTime1+diffTime2," seconds")
            print("Final Accuracy for fold",i+1," is ",acc," Using Optimal hyperparameter C as ", param[i][0]," gamma as ",param[i][1]," r as ",param[i][2]," and degree as ",param[i][3])
            f_Accuracy[k].append(acc)
            
            
print("\n")
print("Final Accuracy for Linear is ",np.sum(f_Accuracy['linear'])/5)   
print("Final Accuracy for Sigmoid is ",np.sum(f_Accuracy['sigmoid'])/5)
print("Final Accuracy for RBF is ",np.sum(f_Accuracy['rbf'])/5)
print("Final Accuracy for Poly is ",np.sum(f_Accuracy['poly'])/5)
            

   

# One vs One with class weight

In [ ]:

ind_list = [i for i in range(214)]
shuffle(ind_list)
train= trainX[ind_list,:]
train_label = trainY[ind_list,]
setSize=(int)(train.shape[0]/5)
f_Accuracy={'linear':[],'rbf':[],'sigmoid':[],'poly':[]}
for i in range(0,5):
    test_set=train[i*setSize:(i+1)*setSize,:]
    test_label=train_label[i*setSize:(i+1)*setSize,]
    train_set=np.delete(train,slice(i*setSize,(i+1)*setSize),0)
    train_lbl=np.delete(train_label,slice(i*setSize,(i+1)*setSize,))
    X_train, X_test, y_train, y_test = train_test_split(train_set, train_lbl, test_size=0.20)
    acc_param=[]
    kernels=['poly']
    Cs = [math.pow(2,-5),math.pow(2,-3),math.pow(2,-1), math.pow(2,0),math.pow(2,1),math.pow(2,3)]
    gammas = [math.pow(2,-15), math.pow(2,-13),math.pow(2,-11),math.pow(2,-9),math.pow(2,-7),math.pow(2,-5),math.pow(2,-3)]
    degree=[2,3,4]
    coeff=[0,0.1,0.01,0.001]
    decision_function_shape='ovo'
    weight="balanced"
    for k in kernels:
        if k=='linear':
            max_acc,optimal_param,diffTime1=linear_svc(Cs,X_train,X_test,y_train,y_test,decision_function_shape,weight)

            acc,param,diffTime2=linear_svc(optimal_param[i],train_set,test_set,train_lbl,test_label,decision_function_shape,weight)
            print("Time taken to find optimal hyperparameters is " ,diffTime1," seconds")
            print("Total time taken i.e. optimal hyperparam plus training using those optimal hyperparameters is ",diffTime1+diffTime2," seconds")
            print("Final Accuracy for fold",i+1," is ",acc," Using optimal hyperparameter C as ", optimal_param[i][0])
            print("\n")
            print("____________________________________")
            print("\n")
            f_Accuracy[k].append(acc)
        elif k=='sigmoid':
            Cs1 = [math.pow(2,-5),math.pow(2,-3),math.pow(2,-1), math.pow(2,0),math.pow(2,1),math.pow(2,3),math.pow(2,5), math.pow(2,8), math.pow(2,11), math.pow(2,13), math.pow(2,15)]
            gammas1 = [math.pow(2,-15), math.pow(2,-13),math.pow(2,-11)]
            max_acc,optimal_param,diffTime1=sigmoid_svc(Cs1,gammas1,coeff,X_train,X_test,y_train,y_test,decision_function_shape)

 
            cc=[optimal_param[i][0]]
            gg=[optimal_param[i][1]]
            rr=[optimal_param[i][2]]
            
            acc,param,diffTime2=sigmoid_svc(cc,gg,rr,train_set,test_set,train_lbl,test_label,decision_function_shape,weight)
            print("Time taken to find optimal hyperparameters is " ,diffTime1," seconds")
            print("Total time taken i.e. optimal hyperparam plus training using those optimal hyperparameters is ",diffTime1+diffTime2," seconds")
            print("Final Accuracy for fold",i+1," is ",acc," Using Optimal hyperparameter C as ", optimal_param[i][0]," gamma as ",optimal_param[i][1],"and r as ",optimal_param[i][2])
            
            print("\n")
            print("____________________________________")
            print("\n")
            
            f_Accuracy[k].append(acc)
        elif k=="rbf":
            max_acc,optimal_param,diffTime1=rbf_svc(Cs,gammas,X_train,X_test,y_train,y_test,decision_function_shape,weight)
  

            cc=[optimal_param[i][0]]
            gg=[optimal_param[i][1]]
            
            
            acc,param,diffTime2=rbf_svc(cc,gg,train_set,test_set,train_lbl,test_label,decision_function_shape,weight)
            print("Time taken to find optimal hyperparameters is " ,diffTime1," seconds")
            print("Total time taken i.e. optimal hyperparam plus training using those optimal hyperparameters is ",diffTime1+diffTime2," seconds")
            print("Final Accuracy for fold",i+1," is ",acc," Using Optimal hyperparameter C as ", optimal_param[i][0]," gamma as ",optimal_param[i][1])
            print("\n")
            print("____________________________________")
            print("\n")

            f_Accuracy[k].append(acc)
        else:
            
            max_acc,optimal_param,diffTime1=poly_svc(Cs,gammas,coeff,degree,X_train,X_test,y_train,y_test,decision_function_shape,weight)
            
          
            print("\n")
            print("____________________________________")
            print("\n")
            
            cc=[optimal_param[i][0]]
            gg=[optimal_param[i][1]]
            rr=[optimal_param[i][2]]
            dd=[optimal_param[i][3]]
            
            acc,param,diffTime2=poly_svc(cc,gg,rr,dd,train_set,test_set,train_lbl,test_label,decision_function_shape,weight)
            print("Time taken to find optimal hyperparameters is " ,diffTime1," seconds")
            print("Total time taken i.e. optimal hyperparam plus training using those optimal hyperparameters is ",diffTime1+diffTime2," seconds")
            print("Final Accuracy for fold",i+1," is ",acc," Using Optimal hyperparameter C as ", param[i][0]," gamma as ",param[i][1]," r as ",param[i][2]," and degree as ",param[i][3])
            f_Accuracy[k].append(acc)
            
            
print("\n")
print("Final Accuracy for Linear is ",np.sum(f_Accuracy['linear'])/5)   
print("Final Accuracy for Sigmoid is ",np.sum(f_Accuracy['sigmoid'])/5)
print("Final Accuracy for RBF is ",np.sum(f_Accuracy['rbf'])/5)
print("Final Accuracy for Poly is ",np.sum(f_Accuracy['poly'])/5)
            

   